### Scoreboard (roc auc scores)

Benchmark score: 0.8185

* Logistic Regression: 
    - h1n1_vaccine: 0.755698562279082
    - seasonal_vaccine: 0.7984074586955308
    - submission: 0.6306 (522 of 2076)
* Random Forest:
    - h1n1_vaccine: 1
    - seasonal_vaccine: 1
    - submission: 0.7312 (506)
* KNN:
    - h1n1_vaccine: 0.9267064398408054
    - seasonal_vaccine: 0.8957474326137169
    - submission: 0.5000

In [88]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [89]:
X_test = pd.read_csv('test_set_features.csv')
X_train = pd.read_csv('training_set_features.csv')
y_train = pd.read_csv('training_set_labels.csv')

In [90]:
print(f"Train data: {X_train.shape}")
print(f"Test data: {X_train.shape}")
print(f"Train labels: {y_train.shape}")

Train data: (26707, 36)
Test data: (26707, 36)
Train labels: (26707, 3)


In [91]:
## Label encode categorical columns
from sklearn import preprocessing
## Encoding categorical variables - age_group, education, race, sex, income_poverty, maritial_status, rent_or_own, 
# employmant_status, employment_industry, employment_occupation
cols = [
    'age_group',
    'education',
    'race',
    'sex',
    'income_poverty',
    'marital_status',
    'rent_or_own',
    'employment_status',
    'employment_industry',
    'employment_occupation',
    'hhs_geo_region',
    'census_msa'
]
le = preprocessing.LabelEncoder()
for col in cols:
    le = le.fit(X_train[col])
    X_train[col] = le.transform(X_train[col])
    X_test[col] = le.transform(X_test[col])
X_train.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2,1,0,1,8,2,0.0,0.0,21,23
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,2,1,1,0,1,0,0.0,0.0,12,19
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,9,0,2.0,0.0,14,21
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,2,1,1,1,5,1,0.0,0.0,21,23
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,0,0,0,0,9,0,1.0,0.0,18,5


In [92]:
X_test.shape

(26708, 36)

In [79]:
# Check for NaN values columns
print(X_train.isnull().any())

respondent_id                  False
h1n1_concern                    True
h1n1_knowledge                  True
behavioral_antiviral_meds       True
behavioral_avoidance            True
behavioral_face_mask            True
behavioral_wash_hands           True
behavioral_large_gatherings     True
behavioral_outside_home         True
behavioral_touch_face           True
doctor_recc_h1n1                True
doctor_recc_seasonal            True
chronic_med_condition           True
child_under_6_months            True
health_worker                   True
health_insurance                True
opinion_h1n1_vacc_effective     True
opinion_h1n1_risk               True
opinion_h1n1_sick_from_vacc     True
opinion_seas_vacc_effective     True
opinion_seas_risk               True
opinion_seas_sick_from_vacc     True
age_group                      False
education                      False
race                           False
sex                            False
income_poverty                 False
m

In [80]:
imputed_X_train = X_train.copy()
imputed_X_test = X_test.copy()

cols_with_missing = (col for col in X_train.columns 
                                 if X_train[col].isnull().any())

for col in cols_with_missing:
    imputed_X_train[col + '_was_missing'] = imputed_X_train[col].isnull()
    imputed_X_test[col + '_was_missing'] = imputed_X_test[col].isnull()

# Imputation
imputer = SimpleImputer()
imputed_X_train = imputer.fit_transform(imputed_X_train)
imputed_X_test = imputer.transform(imputed_X_test)

In [81]:
def train(clf, X_train, y_train, X_test, ids, suffix=''):
    predictions = pd.DataFrame()
    predictions['respondent_id'] = ids
    clf_name = str(clf).strip("()")
    clf.fit(X_train, y_train['h1n1_vaccine'].values)
    print(f"{clf_name} - h1n1_vaccine - {roc_auc_score(y_train['h1n1_vaccine'].values, clf.predict_proba(X_train)[:, 1], average='micro')}")
    predictions['h1n1_vaccine'] = np.float32(clf.predict(X_test))
    clf.fit(X_train, y_train['seasonal_vaccine'].values)
    print(f"{clf_name} - seasonal_vaccine - {roc_auc_score(y_train['seasonal_vaccine'].values, clf.predict_proba(X_train)[:, 1], average='micro')}")
    predictions['seasonal_vaccine'] = np.float32(clf.predict(X_test))
    predictions.to_csv(f"{clf_name} - {suffix}.csv", index=False)

In [82]:
clf = LogisticRegression()  # do scaling and normalization

train(clf, imputed_X_train, y_train, imputed_X_test, X_test.respondent_id)

C:\Users\USer\miniconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression - h1n1_vaccine - 0.755698562279082
LogisticRegression - seasonal_vaccine - 0.7984074586955308


C:\Users\USer\miniconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [83]:
clf = RandomForestClassifier()
train(clf, imputed_X_train, y_train, imputed_X_test, X_test.respondent_id)

RandomForestClassifier - h1n1_vaccine - 1.0
RandomForestClassifier - seasonal_vaccine - 1.0


In [84]:
clf = KNeighborsClassifier(2)
train(clf, imputed_X_train, y_train, imputed_X_test, X_test.respondent_id)

KNeighborsClassifier(n_neighbors=2 - h1n1_vaccine - 0.9267064398408054
KNeighborsClassifier(n_neighbors=2 - seasonal_vaccine - 0.8957474326137169


In [85]:
X_test

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,26707,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,1,1,1,0,7,0,1.0,0.0,1,7
1,26708,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2,1,1,0,1,2,3.0,0.0,1,20
2,26709,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1,0,0,0,5,2,1.0,0.0,10,12
3,26710,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,5,0,1.0,0.0,21,23
4,26711,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0,1,0,0,6,2,0.0,1.0,4,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26703,53410,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,3,2,2,3,2,1,1.0,1.0,21,23
26704,53411,3.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,2,0,1,0,9,2,1.0,3.0,4,18
26705,53412,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2,1,1,1,9,0,1.0,0.0,21,23
26706,53413,3.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0,0,0,1,1,0,1.0,0.0,21,23


In [54]:
clf = XGBClassifier()

predictions = pd.DataFrame()
predictions['respondent_id'] = X_test.respondent_id
clf_name = str(clf).strip("()")
clf.fit(imputed_X_train, y_train['h1n1_vaccine'].values)
preds = clf.predict(imputed_X_test)
preds = [round(value) for value in preds]
print(f"{clf_name} - h1n1_vaccine - {roc_auc_score(y_train['h1n1_vaccine'].values, preds, average='micro')}")
predictions['h1n1_vaccine'] = np.float32(clf.predict(imputed_X_test))
clf.fit(imputed_X_train, y_train['seasonal_vaccine'].values)
preds = clf.predict(imputed_X_test)
preds = [round(value) for value in preds]
print(f"{clf_name} - seasonal_vaccine - {roc_auc_score(y_train['seasonal_vaccine'].values, preds, average='micro')}")
predictions['seasonal_vaccine'] = np.float32(clf.predict(imputed_X_test))
predictions.to_csv(f"{clf_name}{suffix}.csv", index=False)

C:\Users\USer\miniconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:46:59] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


ValueError: Found input variables with inconsistent numbers of samples: [26707, 26708]